## **라이브러리 다운로드 및 불러오기**

In [ ]:
# 라이브러리 다운로드
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git  # 런타임을 다시 시작하라고 하지만, 무시하고 넘기고 진행
!pip install soynlp
!pip install konlpy

In [ ]:
## 라이브러리 불러오기

# 데이터 핸들링
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False  # plot "-" 부호 표기 설정

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format

# 데이터 전처리
import re
from pykospacing import Spacing
from soynlp.normalizer import emoticon_normalize, repeat_normalize
from konlpy.tag import Okt

# 경고 안내 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 시각화 옵션 함수
def plot_option():
    plt.rc('font', family='NanumBarunGothic')
    plt.rcParams["figure.figsize"] = (10,5)
    plt.xticks(rotation=90, size = 12)
    plt.yticks(size = 10)

# 출력 제한 해제
pd.set_option('display.max_row', 200)       # 행(row)
pd.set_option('display.max_columns', 100)   # 열(column)

## **데이터**

In [ ]:
# 데이터 불러오기
MDunpa_df01 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/Fianl_Review_Dataset_01.csv')

# 변수 타입 변환
MDunpa_df01 = MDunpa_df01.astype("str")                                            # '문자' 타입 변환
MDunpa_df01["작성일자(전체)"] = pd.to_datetime(MDunpa_df01["작성일자(전체)"])      # '날자' 타입 변환

# 데이터 전처리
del MDunpa_df01["Unnamed: 0"]

# 데이터 확인
print(MDunpa_df01.shape)
MDunpa_df01.tail()

(11667, 10)


,리뷰_ID,고객_ID,평점,리뷰어,유용성,작성일자(전체),작성일자(년),작성일자(월),작성일자(일),Binary
11662,11662,ᄌᄒ,1,1주년이 뭔 주는게 저번 레압뿌리는거만도못함 그리고 오즈마 횟수는 왜갑자기 증발하는...,1,2023-04-04,2023,4,4,After
11663,11663,YU k,3,사람이 너무 많이 줄었음;; '그 국가'에 정식출시하는 것 말곤 답이 없는 듯,1,2023-04-04,2023,4,4,After
11664,11664,장정현,1,생각이 있다면 버서커 너프좀 하시길,1,2023-04-04,2023,4,4,After
11665,11665,Eddie park,3,오래전 피던에대한 추억 향수으로 잘하고 있지만 이벤트도 너무 숙제같아지고 있고 신규...,1,2023-04-05,2023,4,5,After
11666,11666,롤롤,5,아라드격투왕 터치안되는버그 빨리 고쳐주시고 접속하면 아이템 없다고 뜨는 버그 고쳐줘요,0,2023-04-05,2023,4,5,After


## **EDA 및 전처리**

In [ ]:
## 데이터 전처리

# '유용성' 변수 타입 변환
sample_list01 = []
for i in tqdm(range(0, len(MDunpa_df01))):
  sample_data01 = MDunpa_df01["유용성"][i].replace(',', '')
  sample_list01.append(sample_data01)

del MDunpa_df01["유용성"]
MDunpa_df01["유용성"] = sample_list01

# 일부 변수 타입 변환
MDunpa_df01 = MDunpa_df01.astype({'작성일자(년)': 'int', '작성일자(월)': 'int', '작성일자(일)': 'int', '평점': 'int', '유용성': 'int'})

100%|██████████| 11667/11667 [00:00<00:00, 171835.41it/s]


In [ ]:
# 데이터 정보 확인
MDunpa_df01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11667 entries, 0 to 11666
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   리뷰_ID     11667 non-null  object        
 1   고객_ID     11667 non-null  object        
 2   평점        11667 non-null  int64         
 3   리뷰어       11667 non-null  object        
 4   작성일자(전체)  11667 non-null  datetime64[ns]
 5   작성일자(년)   11667 non-null  int64         
 6   작성일자(월)   11667 non-null  int64         
 7   작성일자(일)   11667 non-null  int64         
 8   Binary    11667 non-null  object        
 9   유용성       11667 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 911.6+ KB


In [ ]:
# '작성일자(전체)' 변수 확인
MDunpa_df01['작성일자(전체)'].value_counts().sort_index(ascending=False)

,count
작성일자(전체),
2023-04-05,2
2023-04-04,5
2023-04-03,6
2023-04-02,7
2023-04-01,6
...,...
2022-03-26,668
2022-03-25,1739
2022-03-24,3486


In [ ]:
# '평점' 변수 확인
MDunpa_df01['평점'].value_counts().sort_index(ascending=False)

,count
평점,
5,6260
4,1290
3,938
2,436
1,2743


In [ ]:
# 'Binary' 변수 확인
MDunpa_df01['Binary'].value_counts().sort_index(ascending=False)

,count
Binary,
Befor,10791
After,876


In [ ]:
# '유용성' 변수 확인
MDunpa_df01['유용성'].value_counts().sort_index(ascending=True)

,count
유용성,
0,10284
1,873
2,201
3,63
4,39
5,25
6,17
7,14
8,10


In [ ]:
# '유용성' 변수를 확인한 결과, 총 1,070개 요인이 존재함을 확인했으며, 대부분의 리뷰들은 유저들에게 공감을 얻지 못했지만, 일부 리뷰들은 많은 공감을 얻는 것을 확인 完
#                              따라서, 100개 이상의 유용성을 인정받은 리뷰 데이터는 분석할 가치가 있다 판단(향후, 탐색해볼 예정)
# '평점' 변수를 확인한 결과, 리뷰가 전반적으로 극단척 선호도를 갖는 것을 확인 完
# 'Binary'변수를 확인한 결과, KGA대상 전/후 데이터 수가 극단적으로 차이가 나는 것을 확인 完

In [ ]:
# 데이터 결측치 확인
MDunpa_df01.isnull().sum()

,0
리뷰_ID,0
고객_ID,0
평점,0
리뷰어,0
작성일자(전체),0
작성일자(년),0
작성일자(월),0
작성일자(일),0
Binary,0
유용성,0


---

### **데이터 전처리**

In [ ]:
# 데이터 확인

print(MDunpa_df01.shape)
MDunpa_df01.tail()

(11667, 10)


,리뷰_ID,고객_ID,평점,리뷰어,작성일자(전체),작성일자(년),작성일자(월),작성일자(일),Binary,유용성
11662,11662,ᄌᄒ,1,1주년이 뭔 주는게 저번 레압뿌리는거만도못함 그리고 오즈마 횟수는 왜갑자기 증발하는...,2023-04-04,2023,4,4,After,1
11663,11663,YU k,3,사람이 너무 많이 줄었음;; '그 국가'에 정식출시하는 것 말곤 답이 없는 듯,2023-04-04,2023,4,4,After,1
11664,11664,장정현,1,생각이 있다면 버서커 너프좀 하시길,2023-04-04,2023,4,4,After,1
11665,11665,Eddie park,3,오래전 피던에대한 추억 향수으로 잘하고 있지만 이벤트도 너무 숙제같아지고 있고 신규...,2023-04-05,2023,4,5,After,1
11666,11666,롤롤,5,아라드격투왕 터치안되는버그 빨리 고쳐주시고 접속하면 아이템 없다고 뜨는 버그 고쳐줘요,2023-04-05,2023,4,5,After,0


In [ ]:
## 함수 생성

# ①전처리 정의: 리뷰어의 공백 제거
def remove_spaces(texts):
    review = re.sub(r'\s+', '', texts)    # 리뷰어 전체공백 제거
    review = re.sub(r"^\s+", '', review)  # 리뷰어 시작부분 공백 제거
    review = re.sub(r'\s+$', '', review)  # 리뷰어 끝부분 공백 제거
    return review


# ②전처리 정의: 리뷰어의 특수문자 제거
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"',
                 '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
def clean_punc(text, punct, punct_mapping):
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    for p in punct:
        text = text.replace(p, f' {p} ')
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    return text.strip()


# ③전처리 정의: 불용어 제거(URL링크, 숫자, 영어, 결측치, 특수문자, 공백 등)
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i]))   #문장부호(punctuation) 제거
        review = re.sub(r'<[^>]+>','',review)        # Hhtml tags 제거
        review = re.sub(r'\d+','', review)           # 숫자 제거
        review = review.upper()                      # 모든 문자를 소문자로 변환
        hangul = re.compile('[^ A-Z-ㄱ-ㅣ가-힣]+')   # 한글, 띄어쓰기, 대문자영어를 제외한 모든 글자 설정
        review = hangul.sub('', review)              # (이어서) 설정 기준 제거
        corpus.append(review)                        # 한 글자씩 불러오므로, 리스트 사용.
        sent = ''.join(corpus)                       # 분리된 리스트 결합
    return sent


## ④전처리 정의: 외래어 사전을 통한 텍스트 변환
# 외래어 사전 위치 설정
loanwords_Dict_map = {}
loanwords_Dict_data = open('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/loanwords_Dict.txt', 'r', encoding='utf-8')
lines = loanwords_Dict_data.readlines()

# 외래어 사전을 불러와 매칭 및 변환
for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]           # 외래어
    ori_word = line.split('\t')[1]             # 외래어에 대한 한국어
    loanwords_Dict_map[miss_spell] = ori_word

In [ ]:
## 한국어 리뷰어 전처리(From. 석사 학위연구를 통해 제안한 방법론)
# 한국어는 모호성과 의역성에 대한 특성을 갖고있어 데이터 전처리 必

print('데이터 전처리 시작(Start Data preprocessing)')
try:
  Review_preprocess01 = []
  spacing = Spacing()

  for i in tqdm(range(0, len(MDunpa_df01))):
    sample_data01 = str(MDunpa_df01["리뷰어"][i])                      # (원본) 리뷰어
    ### 리뷰 데이터가 공백이 아닐 경우,
    if sample_data01 != "" :
      ## 1단계 전처리
      sample_data01 = remove_spaces(sample_data01)                     # ①전처리 함수 적용(리뷰어의 공백 제거)
      sample_data01 = spacing(sample_data01)                           # 한국어 띄어쓰기
      sample_data02 = str(sample_data01)

      ## 2단계 전처리
      sample_data02 = clean_punc(sample_data02, punct, punct_mapping)  # ②전처리 함수 적용(리뷰어의 특수문자 제거)
      sample_data02 = clean_text(sample_data02)                        # ③전처리 함수 적용(불용어 제거)
      sample_data03 = str(sample_data02)

      ## (반복) 1단계 전처리
      sample_data03 = remove_spaces(sample_data03)                     # ①전처리 함수 적용(리뷰어의 공백 제거)
      sample_data03 = spacing(sample_data03)                           # 한국어 띄어쓰기
      sample_data04 = str(sample_data03)

      # 3단계 전처리
      sample_data05 = repeat_normalize(sample_data04, num_repeats=2)   # 텍스트 정규화(자음/모음 일반화)
      for lownword in loanwords_Dict_map:                              # 외래어 처리: 외래어 사전 기반④전처리  함수 적용(외래어 사전을 통한 텍스트 변환)
        sample_data05 = sample_data05.replace(lownword, loanwords_Dict_map[lownword])
      Review_preprocess01.append(sample_data05)

    ### 리뷰 데이터가 공백일 경우,
    else :
      Review_preprocess01.append("공백")
      print(sample_data05)

finally:
  MDunpa_df01["리뷰어02"] = Review_preprocess01

print('데이터 전처리 종료(End Data preprocessing)')

데이터 전처리 시작(Start Data preprocessing)


100%|██████████| 11667/11667 [09:19<00:00, 20.84it/s]

데이터 전처리 종료(End Data preprocessing)


In [ ]:
# 데이터프레임 변수명 변경
MDunpa_df01.columns = ['리뷰_ID', '고객_ID', '평점', '리뷰어(전_전)', '작성일자(전체)', '작성일자(년)', '작성일자(월)', '작성일자(일)', 'Binary', '유용성', '리뷰어(전_후)']

In [ ]:
# 데이터 확인
MDunpa_df01.head()

,리뷰_ID,고객_ID,평점,리뷰어(전_전),작성일자(전체),작성일자(년),작성일자(월),작성일자(일),Binary,유용성,리뷰어(전_후)
0,0,잘들어라 쫌,5,설치한다. 15년전 내가 설레면서 했던 그 갬성이 돌아올지,2022-03-22,2022,3,22,Befor,0,설치한 다년 전 내가 설레면서 했던 그 갬성이 돌아올지
1,1,Minghao Huang,5,"기대하다,드디어 나왓네요.별점5개박고 즐겜시작해보겠습니다.",2022-03-22,2022,3,22,Befor,0,기대하다 드디어 나왓네 요별점 개박고 즐겜 시작해보겠습니다
2,2,노리,5,더도 안바란다. 과거 느낌만 잘 표현해서 향수만 느끼게 해주면 성공한거다...pc던...,2022-03-22,2022,3,22,Befor,0,더 도안 바란다 과거 느낌만 잘 표현해서 향수만 느끼게 해주면 성공한 거다 PC던 ...
3,3,이우영,5,오 기대됩니다 완전 던파 브금.노래등등 추억자극,2022-03-22,2022,3,22,Befor,89,오기대됩니다 완전던 파브금 노래 등 등 추억 자극
4,4,리즈유나이티드,5,"게임성, 과금 모델까지 두루 갖춘 명작",2022-03-22,2022,3,22,Befor,0,게임성과 금 모델까지 두루 갖춘 명작


In [ ]:
# 데이터 백업
#MDunpa_df01.to_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/Fianl_Review_Dataset_02.csv')

---

### **감성분석**

In [ ]:
# 데이터 불러오기
MDunpa_df02 = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/Fianl_Review_Dataset_02.csv')
del MDunpa_df02["Unnamed: 0"]
print(MDunpa_df02.shape)
MDunpa_df02.tail()

(11667, 11)


,리뷰_ID,고객_ID,평점,리뷰어(전_전),작성일자(전체),작성일자(년),작성일자(월),작성일자(일),Binary,유용성,리뷰어(전_후)
11662,11662,ᄌᄒ,1,1주년이 뭔 주는게 저번 레압뿌리는거만도못함 그리고 오즈마 횟수는 왜갑자기 증발하는...,2023-04-04,2023,4,4,After,1,주년이 뭔 주는 게 저번레 압뿌리는 거만도 못함 그리고 오즈마 횟수는 왜 갑자기 증...
11663,11663,YU k,3,사람이 너무 많이 줄었음;; '그 국가'에 정식출시하는 것 말곤 답이 없는 듯,2023-04-04,2023,4,4,After,1,사람이 너무 많이 줄었음 그 국가에 정식 출시하는 것 말곤답이 없는 듯
11664,11664,장정현,1,생각이 있다면 버서커 너프좀 하시길,2023-04-04,2023,4,4,After,1,생각이 있다면 버서 커너프 좀 하시길
11665,11665,Eddie park,3,오래전 피던에대한 추억 향수으로 잘하고 있지만 이벤트도 너무 숙제같아지고 있고 신규...,2023-04-05,2023,4,5,After,1,오래전 피던 에 대한 추억 향수으로 잘하고 있지만 이벤트도 너무 숙제 같아지고 있고...
11666,11666,롤롤,5,아라드격투왕 터치안되는버그 빨리 고쳐주시고 접속하면 아이템 없다고 뜨는 버그 고쳐줘요,2023-04-05,2023,4,5,After,0,아라드격투왕 터치 안 되는 버그빨리 고쳐주시고 접속하면 아이템 없다고 뜨는 버그 고쳐줘요


In [ ]:
## 감성분석
# 1단계: 감성분석을 위한 4개 품사 추출 리스트 정의

clean_04_words_list = []
clean_01_words_list = []

for i in tqdm(range(0, len(MDunpa_df02))):
    okt = Okt()                                                      # Okt형태소 분석기 정의(다른 형태소 분석기 보다 오타를 잡아주기 때문에 온라인 리뷰에 적절한 형태소라 판단)
    clean_04_words = []                                              # 감성분석을 위한 리스트(4개 품사 저장)
    clean_01_words = []                                              # 빈도분석을 위한 리스트(1개 품사 저장 리스트)

    sample_data01 = okt.pos(str(MDunpa_df02["리뷰어(전_후)"][i]))    # 형태소 분식기를 통한 '단어/품사명' 출력

    for word in sample_data01:                                       # 리뷰어의 토큰을 출력하여 조건문에 기입
        if word[1] in ['Noun', 'Verb', 'Adjective', 'Adverb']:       # 4품사 필터링 설정: ①명사(명사, 대명사, 고유명사), ②동사, ③형용사, ④부사
            clean_04_words.append(word[0])
    clean_04_words_list.append(clean_04_words)

    for word in sample_data01:
        if word[1] in ['Noun']:
            clean_01_words.append(word[0])
    clean_01_words_list.append(clean_01_words)

# 감성분석에 활용한 형태소 백업
MDunpa_df02['품사(KNU)']  = clean_04_words_list
MDunpa_df02['품사(freq)'] = clean_01_words_list

100%|██████████| 11667/11667 [00:41<00:00, 278.08it/s]


In [ ]:
## 감성분석
# 2단계: 불용어사전 및 1개 글자 수를 갖는 단어 제거

# 불용어사전 정의
stopwords_list = []
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

for i in range(0, len(stopwords)) :
    sample_data02 = stopwords[i][0]
    stopwords_list.append(sample_data02)
print("불용아 사전 단어 수: ", end=""); print(len(stopwords_list))

# 감성분석에 불필요한 불용어 제거: 불용어사전 중심으로
sample_review_list02 = []
for i in tqdm(range(0, len(MDunpa_df02))) :
    sample_review_list01 = []

    sample_data03 = MDunpa_df02["품사(KNU)"][i]           # '품사(KNU)' 변수 불러오기

    for j in sample_data03 :                              # 불용어사전을 통한 필터링(토큰 길이가 '1'인 텍스트 제거)
        if j in stopwords_list :
            pass
        else :
            if len(j) == 1 :
                pass
            else :
                sample_review_list01.append(j)
    sample_review_list02.append(sample_review_list01)
print(len(sample_review_list02))

# 불용어가 제거된 형태소 백업
MDunpa_df02['품사(KNU)2'] = sample_review_list02

불용아 사전 단어 수: 674


100%|██████████| 11667/11667 [00:01<00:00, 7326.02it/s]

11667


In [ ]:
# 감성분석을 위한 전처리 결과 비교

print(MDunpa_df02['품사(KNU)'][10000])
print(MDunpa_df02['품사(KNU)2'][10000])

['리뷰', '워', '버리노']
['리뷰', '버리노']


___

### **KNU감성사전 구축(리스트 형태)**

In [ ]:
## 감성분석
# 3단계: KNU감성사전 구축

# KNU감성사전 구축
KNU_Dic = pd.read_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/KNU_Dict.txt', sep = "\t", engine='python')   # KNU감성사전 불러오기
KNU_Dictionary = KNU_Dic[:]

KNU_Word = []; KNU_Number = []; KNU_Word_Number = []

for i in tqdm(range(0, len(KNU_Dictionary["문자"]))) :
    KNU_Word.append(KNU_Dictionary["문자"][i])
    KNU_Number.append(KNU_Dictionary["점수"][i])
    KNU_Word_Number.append(str(KNU_Dictionary["문자"][i]) + "." + str(KNU_Dictionary["점수"][i]))

Raw_Data = {'단어': KNU_Word,
            '감성점수': KNU_Number,
            '단어_감성점수': KNU_Word_Number}
KNU_Dictionary_Df = pd.DataFrame(Raw_Data)

# 감성사전 형태 수정
KNU_Dictionary_Df02 = KNU_Dictionary_Df["단어_감성점수"]
Final_fKNU_Dictionary = KNU_Dictionary_Df02.str.split('.')

100%|██████████| 14855/14855 [00:00<00:00, 47970.76it/s]


In [ ]:
Final_fKNU_Dictionary

,단어_감성점수
0,"[(-;, 1]"
1,"[(;_;), -1]"
2,"[(^^), 1]"
3,"[(^-^), 1]"
4,"[(^^*, 1]"
...,...
14850,"[갈등, -1]"
14851,"[의혹, -1]"
14852,"[내팽개치다, -2]"
14853,"[횡령, -2]"


In [ ]:
## 감성분석
# 4단계: 감성점수 및 극성 측정

dic_count = 0
reviews_number = []
senti_number = []
overall_senti_number = []

for i in tqdm(range(0, len(MDunpa_df02))) :
    sample_data04 = MDunpa_df02["품사(KNU)2"][i]

    for j in range(0, len(KNU_Dictionary_Df)) :
        sample_data05 = Final_fKNU_Dictionary[j][0]  # KNU감성사전의 단어
        sample_data06 = Final_fKNU_Dictionary[j][1]  # KNU감성사전의 단어점수

        # 리뷰어의 토큰 중에 감성사전의 토큰과 공통된 단어가 있다면 감성점수 측정
        if sample_data05 in sample_data04 :
            dic_count = dic_count + (int(sample_data06))
        else :
            dic_count = dic_count + (0)

    # 감성점수 삽입
    reviews_number.append(dic_count)

    # 감성극성 측정
    if dic_count > 0:                        # 긍정 리뷰(2)
        overall_senti_number.append("긍정")
    elif dic_count < 0:                      # 부정 리뷰(0)
        overall_senti_number.append("부정")
    else :                                   # 나머지(중립 리뷰)(1)
        overall_senti_number.append("중립")
    dic_count = 0

print(len(reviews_number))
MDunpa_df02["감성점수(KNU)"] = reviews_number
MDunpa_df02["감성극성(KNU)"] = overall_senti_number

100%|██████████| 11667/11667 [13:19<00:00, 14.59it/s]

11667


In [ ]:
## 데이터 전처리

# 데이터 변수 제거 및 변수명 변경
del MDunpa_df02["품사(KNU)"]
MDunpa_df02.columns = ['리뷰_ID', '고객_ID', '평점', '(원본)리뷰어', '작성일자', '년', '월', '일',  'Binary', '유용성', '(정제)리뷰어', '품사(빈도)', '품사(감성)', '감성점수', '감성극성']


# '평점극성' 변수 생성
sample_data01_list = []
for i in range(0, len(MDunpa_df02)):
  sample_data01 = MDunpa_df02["평점"][i]
  if sample_data01 > 3:
    sample_data01_list.append("긍정")  # 4, 5
  elif sample_data01 == 3:
    sample_data01_list.append("중립")  # 3
  else :
    sample_data01_list.append("부정")  # 1, 2

MDunpa_df02["평점극성"] = sample_data01_list


# 데이터프레임 변수 순서변경
MDunpa_df02 = MDunpa_df02[['리뷰_ID', '고객_ID', '평점', '평점극성', '감성점수', '감성극성', '유용성', '(원본)리뷰어', '(정제)리뷰어', '품사(빈도)', '품사(감성)', '작성일자', '년', '월', '일', 'Binary']]

In [ ]:
MDunpa_df02.head(2)

,리뷰_ID,고객_ID,평점,평점극성,감성점수,감성극성,유용성,(원본)리뷰어,(정제)리뷰어,품사(빈도),품사(감성),작성일자,년,월,일,Binary
0,0,잘들어라 쫌,5,긍정,0,중립,0,설치한다. 15년전 내가 설레면서 했던 그 갬성이 돌아올지,설치한 다년 전 내가 설레면서 했던 그 갬성이 돌아올지,"[설치, 다년, 전, 내, 그, 성]","[설치, 다년, 설레면서, 했던, 돌아올지]",2022-03-22,2022,3,22,Befor
1,1,Minghao Huang,5,긍정,1,긍정,0,"기대하다,드디어 나왓네요.별점5개박고 즐겜시작해보겠습니다.",기대하다 드디어 나왓네 요별점 개박고 즐겜 시작해보겠습니다,"[나왓, 점, 개, 즐겜, 시작]","[기대하다, 드디어, 나왓, 박고, 즐겜, 시작, 해보겠습니다]",2022-03-22,2022,3,22,Befor


In [ ]:
MDunpa_df02['평점극성'].value_counts().sort_index(ascending=True)

,count
평점극성,
긍정,7550
부정,3179
중립,938


In [ ]:
MDunpa_df02['감성극성'].value_counts().sort_index(ascending=True)

,count
감성극성,
긍정,2173
부정,1510
중립,7984


In [ ]:
# 상반된 선호도 리뷰 제거(평점극성과 감성극성을 비교)

rating_list = []
for i in tqdm(range(0, len(MDunpa_df02))):
    star_rating = MDunpa_df02["평점극성"][i]
    senti_rating = MDunpa_df02["감성극성"][i]

    if star_rating == "긍정" and senti_rating == "부정":
        number_values = "불일치"
    elif star_rating == "부정" and senti_rating == "긍정":
        number_values = "불일치"
    else :
        number_values = "일치"
    rating_list.append(number_values)
MDunpa_df02["filtering"] = rating_list

100%|██████████| 11667/11667 [00:00<00:00, 97978.45it/s]


In [ ]:
MDunpa_df02['filtering'].value_counts().sort_index(ascending=True)

,count
filtering,
불일치,1014
일치,10653


In [ ]:
# 데이터 백업
#MDunpa_df02.to_csv('/content/drive/MyDrive/3. Research/[기업] 분석 과제/네오플 트렌드 분석/Data/Fianl_Review_Dataset_03.csv')

___